<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: Unit Testing `GiRaFFE_NRPy`: $\tilde{S}_i$ source term, $A_i$ gauge term, and $\psi^6 \Phi$

## Author: Patrick Nelson
### Formatting improvements courtesy Brandon Clark

## This module Validates the `Stilde_source_A_gauge_Phi_rhs` routine for `GiRaFFE`.

**Notebook Status:** <font color='red'><b>In Progress</b></font>

**Validation Notes:** This module will validate the routines in [Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs](Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.ipynb).

### NRPy+ Source Code for this module: 
* [GiRaFFE_NRPy/Stilde_source_A_gauge_Phi_rhs.py](../../edit/in_progress/GiRaFFE_NRPy/Stilde_source_A_gauge_Phi_rhs.py) [\[**tutorial**\]](Tutorial-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs.ipynb) Generates the driver to compute the magnetic field from the vector potential in arbitrary spactimes.

## Introduction:

This notebook validates the code that will add simpler terms to the RHSs of our evolved, conservative variables in `GiRaFFE_NRPy`. These terms require only basic finite-differencing for their derivatives, so the code to generate them is far simpler.

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. We will generate test data with arbitrarily-chosen analytic functions and calculate gridfunctions at the cell centers on a small numeric grid. We will then run the algorithms to compute the RHS terms numerically along with similar algorithms to compute them analytically, for comparison. We will then compare the data output by both routines to show convergence 


<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#setup): Set up core functions and parameters for unit testing the algorithm
    1. [Step 1.a](#metric_expressions) Write expressions for the metric gridfunctions
    1. [Step 1.b](#mhd_expressions) Write expressions for the magnetohydrodynamic gridfunctions
    1. [Step 1.c](#ccodekernels) Generate C functions to calculate the gridfunctions
    1. [Step 1.d](#analytic_comparison) Generate C code to analytically calculate RHSs
    1. [Step 1.e](#functions_to_test) Generate C code to numerically calculate RHSs
    1. [Step 1.f](#free_parameters) Set free parameters in the code
1. [Step 2](#mainc): `Stilde_source_A_gauge_Phi_rhs_unit_test.c`: The Main C Code
    1. [Step 2.a](#compile_run): Compile and run the code
1. [Step 3](#convergence): Code validation: Verify that relative error in numerical solution converges to zero at the expected order
1. [Step 4](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='setup'></a>

# Step 1: Set up core functions and parameters for unit testing the algorithm \[Back to [top](#toc)\]
$$\label{setup}$$

We'll start by appending the relevant paths to `sys.path` so that we can access sympy modules in other places. Then, we'll import NRPy+ core functionality and set up a directory in which to carry out our test. 

In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir = "Validation/"
cmd.mkdir(out_dir)

thismodule = "Start_to_Finish-GiRaFFE_NRPy-Stilde_source_A_gauge_Phi_rhs"

# Set the finite-differencing order to 2
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 2)


<a id='metric_expressions'></a>

## Step 1.a: Write expressions for the metric gridfunctions \[Back to [top](#toc)\]
$$\label{metric_expressions}$$

Now, we'll choose some functions with arbitrary forms to generate test data. We'll need to set ten gridfunctions, so expressions are being pulled from several previously written unit tests.

\begin{align}
\gamma_{xx} &= ax^3 + by^3 + cz^3 + dy^2 + ez^2 + f \\
\gamma_{yy} &= gx^3 + hy^3 + lz^3 + mx^2 + nz^2 + p \\
\gamma_{zz} &= px^3 + qy^3 + rz^3 + sx^2 + ty^2 + u. \\
\gamma_{xy} &= a \exp\left(-\left((x-b)^2+(y-c)^2+(z-d)^2\right)\right) \\
\gamma_{xz} &= f \exp\left(-\left((x-g)^2+(y-h)^2+(z-l)^2\right)\right) \\
\gamma_{yz} &= m \exp\left(-\left((x-n)^2+(y-o)^2+(z-p)^2\right)\right), \\
\beta^x &= \arctan(ax + by + cz) \\
\beta^y &= \arctan(bx + cy + az) \\
\beta^z &= \arctan(cx + ay + bz) \\
\alpha &= 1 - \frac{1}{2+x^2+y^2+z^2} \\
\end{align}


In [2]:
a,b,c,d,e,f,g,h,l,m,n,o,p,q,r,s,t,u = par.Cparameters("REAL",thismodule,["a","b","c","d","e","f","g","h","l","m","n","o","p","q","r","s","t","u"],1e300)
M_PI  = par.Cparameters("#define",thismodule,["M_PI"], "")

gammaDD = ixp.register_gridfunctions_for_single_rank2("AUXEVOL","gammaDD","sym01",DIM=3)
betaU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","betaU",DIM=3)
alpha = gri.register_gridfunctions("AUXEVOL","alpha")

par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

gammaDD[0][0] = a*x**3 + b*y**3 + c*z**3 + d*y**2 + e*z**2 + f
gammaDD[1][1] = g*x**3 + h*y**3 + l*z**3 + m*x**2 + n*z**2 + o
gammaDD[2][2] = p*x**3 + q*y**3 + r*z**3 + s*x**2 + t*y**2 + u
gammaDD[0][1] = a * sp.exp(-((x-b)**2 + (y-c)**2 + (z-d)**2))
gammaDD[0][2] = f * sp.exp(-((x-g)**2 + (y-h)**2 + (z-l)**2))
gammaDD[1][2] = m * sp.exp(-((x-n)**2 + (y-o)**2 + (z-p)**2))

betaU[0] = sp.atan(a*x + b*y + c*z)
betaU[1] = sp.atan(b*x + c*y + a*z)
betaU[2] = sp.atan(c*x + a*y + b*z)

alpha = sp.sympify(1.0) - sp.sympify(1.0) / (sp.sympify(2.0) + x**2 + y**2 + z**2)

<a id='mhd_expressions'></a>

## Step 1.b: Write expressions for the magnetohydrodynamic gridfunctions \[Back to [top](#toc)\]
$$\label{mhd_expressions}$$

We will also pick arbitrary analytic forms for the vector potential and Valencia three-velocity here. 
\begin{align}
A_x &= \exp(ey+fz) \\
A_y &= \exp(fz+dx) \\
A_z &= \exp(dx+ey) \\
\bar{v}^x &= \frac{2}{\pi} \arctan(ax + by + cz) \\
\bar{v}^y &= \frac{2}{\pi} \arctan(bx + cy + az) \\
\bar{v}^z &= \frac{2}{\pi} \arctan(cx + ay + bz) \\
\end{align}

We compute the magnetic field from the vector potential analytically as $B^i = \epsilon^{ijk} A_{k,j}$. Additionally, we let the scalar potential $\sqrt{\gamma} \Phi$ be defined as $\sqrt{\gamma} \left(1+x^2+y^2+z^2\right)$, where $\gamma$ is the determinant of the three metric $\gamma_{ij}$

In [3]:
AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
BU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","BU")
ValenciavU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","ValenciavU")

AD[0] = sp.exp(e*y+f*z)
AD[1] = sp.exp(f*z+d*x)
AD[2] = sp.exp(d*x+e*y)
ValenciavU[0] = (sp.sympify(2.0)/M_PI) * sp.atan(d*x + e*y + f*z)
ValenciavU[1] = (sp.sympify(2.0)/M_PI) * sp.atan(e*x + f*y + d*z)
ValenciavU[2] = (sp.sympify(2.0)/M_PI) * sp.atan(f*x + d*y + e*z)

import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LCijk = LeviCivitaDDD[i][j][k]
            #LeviCivitaDDD[i][j][k] = LCijk * sp.sqrt(gho.gammadet)
            LeviCivitaUUU[i][j][k] = LCijk / sp.sqrt(gammadet)

BU = ixp.zerorank1() # BU is already registered as a gridfunction, but we need to zero its values and declare it in this scope.

for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            BU[i] += LeviCivitaUUU[i][j][k] * sp.diff(AD[k],rfm.xxCart[j])

gammaUU,gammaDET = ixp.symm_matrix_inverter3x3(gammaDD)
psi6Phi = gri.register_gridfunctions("EVOL","psi6Phi")
psi6Phi = gammaDET * (1+x*x+y*y+z*z)

<a id='ccodekernels'></a>

## Step 1.c: Generate C functions to calculate the gridfunctions \[Back to [top](#toc)\]
$$\label{ccodekernels}$$

We now use NRPy+'s `outCfunction()` to generate the functions that will calculate our sample data on numerical grids.

In [4]:
metric_gfs_to_print = [\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD00"),rhs=gammaDD[0][0]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD01"),rhs=gammaDD[0][1]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD02"),rhs=gammaDD[0][2]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD11"),rhs=gammaDD[1][1]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD12"),rhs=gammaDD[1][2]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","gammaDD22"),rhs=gammaDD[2][2]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","betaU0"),rhs=betaU[0]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","betaU1"),rhs=betaU[1]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","betaU2"),rhs=betaU[2]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","alpha"),rhs=alpha),\
                      ]
desc = "Calculate the metric gridfunctions"
name = "calculate_metric_gfs"
outCfunction(
    outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *restrict params,REAL *restrict xx[3],REAL *restrict auxevol_gfs",
    body     = fin.FD_outputC("returnstring",metric_gfs_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts ="AllPoints,Read_xxs")

EM_to_print = [\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","BU0"),rhs=BU[0]),\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","BU1"),rhs=BU[1]),\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","BU2"),rhs=BU[2]),\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","AD0"),rhs=BU[0]),\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","AD1"),rhs=BU[1]),\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","AD2"),rhs=BU[2]),\
               lhrh(lhs=gri.gfaccess("auxevol_gfs","psi6Phi"),rhs=psi6Phi),\
              ]

desc = "Calculate sample magnetic field and potential data"
name = "calculate_BU_AD_psi6Phi"
outCfunction(
    outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *params,REAL *xx[3],REAL *auxevol_gfs",
    body     = fin.FD_outputC("returnstring",EM_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts ="AllPoints,Read_xxs")

ValenciavU_to_print = [\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","ValenciavU0"),rhs=ValenciavU[0]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","ValenciavU1"),rhs=ValenciavU[1]),\
                       lhrh(lhs=gri.gfaccess("auxevol_gfs","ValenciavU2"),rhs=ValenciavU[2]),\
                      ]

desc = "Calculate sample velocity data"
name = "calculate_ValenciavU"
outCfunction(
    outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
    params   ="const paramstruct *params,REAL *xx[3],REAL *auxevol_gfs",
    body     = fin.FD_outputC("returnstring",ValenciavU_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts ="AllPoints,Read_xxs")


Output C function calculate_metric_gfs() to file Validation/calculate_metric_gfs.h
Output C function calculate_BU() to file Validation/calculate_BU.h
Output C function calculate_ValenciavU() to file Validation/calculate_ValenciavU.h


<a id='analytic_comparison'></a>

## Step 1.d: Generate C code to analytically calculate RHSs \[Back to [top](#toc)\]
$$\label{analytic_comparison}$$

In order to do a unit test, we have to have some expected value against which to compare. In this case, we'll calculate the expected output of our functions using the analytic forms of the gridfunction defined above. The first step to doing this will be to analytically compute the derivatives that we will need. For the $\partial_t \tilde{S}_i$ source term, this will just be the metric derivatives. 

In [ ]:
gammaDD_dD = ixp.zerorank3()
for i in range(3):
    for j in range(3):
        for k in range(3):
            gammaDD_dD[i][j][k] = sp.diff(gammaDD[i][j],rfm.xxCart[k])
            
betaU_dD = ixp.zerorank2()
for i in range(3):
    for j in range(3):
        betaU_dD[i][j] = sp.diff(betaU[i],rfm.xxCart[j])
        
alpha_dD = ixp.zerorank1()
for i in range(3):
    alpha_dD[i] = sp.diff(alpha,rfm.xxCart[i])
    
import GRHD.equations as GRHD
    
GRHD.u4U_in_terms_of_ValenciavU__rescale_ValenciavU_by_applying_speed_limit(alpha,betaU,gammaDD, ValenciavU)

# Next sqrt(gamma)
GRHD.compute_sqrtgammaDET(gammaDD)

# Then compute g4DD_zerotimederiv_dD
GRHD.compute_g4DD_zerotimederiv_dD(gammaDD,betaU,alpha, gammaDD_dD,betaU_dD,alpha_dD)

# small b
GRFFE.compute_smallb4U(gammaDD,betaU,alpha, GRHD.u4U_ito_ValenciavU, BU, sqrt4pi)
GRFFE.compute_smallbsquared(gammaDD, betaU, alpha, GRFFE.smallb4U)
# Electromagnetic stress-energy tensor
GRFFE.compute_TEM4UU(gammaDD,betaU,alpha, GRFFE.smallb4U, GRFFE.smallbsquared,GRHD.u4U_ito_ValenciavU)

# Add the source term to the RHS
Stilde_rhsD = ixp.zerorank1(DIM=3)
for i in range(3):
    for mu in range(4):
        for nu in range(4):
            # \frac{1}{2} \alpha \sqrt{\gamma} T^{\mu \nu}_{\rm EM} \partial_i g_{\mu \nu}
            Stilde_rhsD[i] += sp.Rational(1,2) * alpha * GRHD.sqrtgammaDET * \
                              GRFFE.TEM4UU[mu][nu] * GRHD.g4DD_zerotimederiv_dD[mu][nu][i+1]


Next, we will handle the $\partial_t A_i$ gauge term, $-\partial_i (\alpha \Phi - \beta^j A_j).$ Carrying through this differentiation analytically, we see that we will need to code 
$$
-\alpha_{,i} \Phi - \alpha \Phi_{,i} + \beta^j_{,i} A_j + \beta^j A_{j,i}
$$

In [ ]:
psi6Phi_dD = ixp.zerorank1()
for i in range(3):
    psi6Phi_dD[i] = sp.diff(psi6Phi,rfm.xxCart[i])
AD_dD = ixp.zerorank2()
for i in range(3):
    for j in range(3):
        AD_dD[i][j] = sp.diff(AD[i],rfm.xxCart[j])

A_rhsD = ixp.zerorank1()
for i in range(3):
    A_rhsD[i] += -alpha_dD[i]*psi6Phi/sp.sqrt(gammaDET) - alpha*psi6Phi_dD[i]
    
for i in range(3):
    for j in range(3):
        A_rhsD[i] += betaU_dD[j][i]*AD[j] + betaU[j]*AD[j][i]


<a id='functions_to_test'></a>

## Step 1.e: Generate C code to numerically calculate RHSs \[Back to [top](#toc)\]
$$\label{functions_to_test}$$

Here, we will generate the functions that we specifically wish to test. We'll start with the source term for $\partial_t \tilde{S}_i$. This term involves derivatives of the metric tensor, and in `GiRaFFE`, we will be storing interpolations of the metric quantities to the cell faces of our grid. This will allow us to save some time in our simulations if we compute the finite differences in a somewhat unusual way. If we do a simple, first-order finite-difference derivative of these interpolations, we can take a derivative with the accuracy of a higher-order stencil but the computational cost of a first-order stencil. To do this, we will create arrays of NRPy+'s `Cparameter`s to pass to the function that will build the RHS, then write the lines of code by hand to compute the derivatives for them. 

Additionally, we will write this as three similar functions, corresponding to once in each spatial direction as we take derivatives. 

In [5]:
import GiRaFFE_NRPy.Stilde_source_A_gauge_Phi_rhs as rhs

# Reset gridfunctions to basenames: 
gammaDD = ixp.declarerank2("gammaDD","sym01",DIM=3)
betaU = ixp.declarerank1("betaU",DIM=3)
alpha = sp.symbols("alpha",real=True)
ValenciavU = ixp.declarerank1("ValenciavU",DIM=3)
BU = ixp.declarerank1("BU",DIM=3)
AD = ixp.declarerank1("AD",DIM=3)
psi6Phi = sp.symbols("psi6Phi",real=True)

# Declare the gridfunction to which we will write
StildeD = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","StildeD")

# Declare all the Cparameterse we will need
sqrt4pi = par.Cparameters("REAL",thismodule,"sqrt4pi","sqrt(4.0*M_PI)")
metricderivDDD = ixp.declarerank3("metricderivDDD","sym01",DIM=3)
shiftderivUD = ixp.declarerank2("shiftderivUD","nosym",DIM=3)
lapsederivD = ixp.declarerank1("lapsederivD",DIM=3)

general_access = """const REAL gammaDD00 = auxevol_gfs[IDX4S(GAMMADD00GF,i0,i1,i2)];
const REAL gammaDD01 = auxevol_gfs[IDX4S(GAMMADD01GF,i0,i1,i2)];
const REAL gammaDD02 = auxevol_gfs[IDX4S(GAMMADD02GF,i0,i1,i2)];
const REAL gammaDD11 = auxevol_gfs[IDX4S(GAMMADD11GF,i0,i1,i2)];
const REAL gammaDD12 = auxevol_gfs[IDX4S(GAMMADD12GF,i0,i1,i2)];
const REAL gammaDD22 = auxevol_gfs[IDX4S(GAMMADD22GF,i0,i1,i2)];
const REAL betaU0 = auxevol_gfs[IDX4S(BETAU0GF,i0,i1,i2)];
const REAL betaU1 = auxevol_gfs[IDX4S(BETAU1GF,i0,i1,i2)];
const REAL betaU2 = auxevol_gfs[IDX4S(BETAU2GF,i0,i1,i2)];
const REAL alpha = auxevol_gfs[IDX4S(ALPHA,i0,i1,i2)];
const REAL ValenciavU0 = auxevol_gfs[IDX4S(VALENCIAVU0GF,i0,i1,i2)];
const REAL ValenciavU1 = auxevol_gfs[IDX4S(VALENCIAVU1GF,i0,i1,i2)];
const REAL ValenciavU2 = auxevol_gfs[IDX4S(VALENCIAVU2GF,i0,i1,i2)];
const REAL BU0 = auxevol_gfs[IDX4S(BU0GF,i0,i1,i2)];
const REAL BU1 = auxevol_gfs[IDX4S(BU1GF,i0,i1,i2)];
const REAL BU2 = auxevol_gfs[IDX4S(BU2GF,i0,i1,i2)];
"""
metric_deriv_access = ixp.zerorank1(DIM=3)
metric_deriv_access[0] = """const REAL metricderivDDD000 = (auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1,i2)])/dxx0;
const REAL metricderivDDD010 = (auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0,i1,i2)])/dxx0;
const REAL metricderivDDD020 = (auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0,i1,i2)])/dxx0;
const REAL metricderivDDD110 = (auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0,i1,i2)])/dxx0;
const REAL metricderivDDD120 = (auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0,i1,i2)])/dxx0;
const REAL metricderivDDD220 = (auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0,i1,i2)])/dxx0;
const REAL shiftderivUD00 = (auxevol_gfs[IDX4S(BETA_FACEU0GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(BETA_FACEU0GF,i0,i1,i2)])/dxx0;
const REAL shiftderivUD10 = (auxevol_gfs[IDX4S(BETA_FACEU1GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(BETA_FACEU1GF,i0,i1,i2)])/dxx0;
const REAL shiftderivUD20 = (auxevol_gfs[IDX4S(BETA_FACEU2GF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(BETA_FACEU2GF,i0,i1,i2)])/dxx0;
const REAL lapsederivUD20 = (auxevol_gfs[IDX4S(ALPHA_FACEGF,i0+1,i1,i2)]-auxevol_gfs[IDX4S(ALPHA_FACEGF,i0,i1,i2)])/dxx0;
"""
metric_deriv_access[1] = """const REAL metricderivDDD001 = (auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1,i2)])/dxx1;
const REAL metricderivDDD011 = (auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0,i1,i2)])/dxx1;
const REAL metricderivDDD021 = (auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0,i1,i2)])/dxx1;
const REAL metricderivDDD111 = (auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0,i1,i2)])/dxx1;
const REAL metricderivDDD121 = (auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0,i1,i2)])/dxx1;
const REAL metricderivDDD221 = (auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0,i1,i2)])/dxx1;
const REAL shiftderivUD01 = (auxevol_gfs[IDX4S(BETA_FACEU0GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(BETA_FACEU0GF,i0,i1,i2)])/dxx1;
const REAL shiftderivUD11 = (auxevol_gfs[IDX4S(BETA_FACEU1GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(BETA_FACEU1GF,i0,i1,i2)])/dxx1;
const REAL shiftderivUD21 = (auxevol_gfs[IDX4S(BETA_FACEU2GF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(BETA_FACEU2GF,i0,i1,i2)])/dxx1;
const REAL lapsederivUD21 = (auxevol_gfs[IDX4S(ALPHA_FACEGF,i0,i1+1,i2)]-auxevol_gfs[IDX4S(ALPHA_FACEGF,i0,i1,i2)])/dxx1;
"""
metric_deriv_access[2] = """const REAL metricderivDDD001 = (auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(GAMMA_FACEDD00GF,i0,i1,i2)])/dxx2;
const REAL metricderivDDD012 = (auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(GAMMA_FACEDD01GF,i0,i1,i2)])/dxx2;
const REAL metricderivDDD022 = (auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(GAMMA_FACEDD02GF,i0,i1,i2)])/dxx2;
const REAL metricderivDDD112 = (auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(GAMMA_FACEDD11GF,i0,i1,i2)])/dxx2;
const REAL metricderivDDD122 = (auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(GAMMA_FACEDD12GF,i0,i1,i2)])/dxx2;
const REAL metricderivDDD222 = (auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(GAMMA_FACEDD22GF,i0,i1,i2)])/dxx2;
const REAL shiftderivUD02 = (auxevol_gfs[IDX4S(BETA_FACEU0GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(BETA_FACEU0GF,i0,i1,i2)])/dxx2;
const REAL shiftderivUD12 = (auxevol_gfs[IDX4S(BETA_FACEU1GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(BETA_FACEU1GF,i0,i1,i2)])/dxx2;
const REAL shiftderivUD22 = (auxevol_gfs[IDX4S(BETA_FACEU2GF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(BETA_FACEU2GF,i0,i1,i2)])/dxx2;
const REAL lapsederivUD22 = (auxevol_gfs[IDX4S(ALPHA_FACEGF,i0,i1,i2+1)]-auxevol_gfs[IDX4S(ALPHA_FACEGF,i0,i1,i2)])/dxx2;
"""
write_final_quantity = ixp.zerorank1(DIM=3)
write_final_quantity[0] = """rhs_gfs[IDX4S(STILDED0GF,i0,i1,i2)] += Stilde_rhsD0;
"""
write_final_quantity[1] = """rhs_gfs[IDX4S(STILDED1GF,i0,i1,i2)] += Stilde_rhsD1;
"""
write_final_quantity[2] = """rhs_gfs[IDX4S(STILDED2GF,i0,i1,i2)] += Stilde_rhsD2;
"""
# Finally, we can write out the files
rhs.compute_StildeD_source_term(gammaDD,betaU,alpha,metricderivDDD,shiftderivUD,lapsederivD,sqrt4pi,ValenciavU,BU)
for i in range(3):
    desc = "Adds the source term to StildeD"+str(i)+"."
    name = "calculate_StildeD"+str(i)+"_source_term"
    outCfunction(
        outfile  = os.path.join(out_dir,name+".h"), desc=desc, name=name,
        params   ="const paramstruct *params,REAL *xx[3],const REAL *auxevol_gfs, REAL *rhs_gfs",
        body     = general_access \
                  +metric_deriv_access[i]\
                  +outputC(rhs.Stilde_rhsD[i],"Stilde_rhsD"+str(i),"returnstring",params="outCverbose=False").replace("IDX4","IDX4S")\
                  +write_final_quantity[i],
        loopopts ="InteriorPoints")


Output C function calculate_StildeD0_source_term() to file Validation/calculate_StildeD0_source_term.h
Output C function calculate_StildeD1_source_term() to file Validation/calculate_StildeD1_source_term.h
Output C function calculate_StildeD2_source_term() to file Validation/calculate_StildeD2_source_term.h


<a id='free_parameters'></a>

## Step 1.f: Set free parameters in the code \[Back to [top](#toc)\]
$$\label{free_parameters}$$


<a id='mainc'></a>

# Step 2: `Stilde_source_A_gauge_Phi_rhs_unit_test.c`: The Main C Code \[Back to [top](#toc)\]
$$\label{mainc}$$


<a id='convergence'></a>

# Step 3: Code validation: Verify that relative error in numerical solution converges to zero at the expected order \[Back to [top](#toc)\]
$$\label{convergence}$$


<a id='latex_pdf_output'></a>

# Step 4: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$
